In [1]:
import numpy as np

## Defining the enviroment

In [2]:
area_rows = 11
area_columns = 11

In [3]:
q_table = np.zeros((area_rows, area_columns, 4))

In [4]:
actions = ['up', 'right', 'down', 'left']

In [5]:
rewards = np.full((area_rows, area_columns), -100)

In [6]:
paths = {} 
paths[0] = [3,4,5,6,7,10]
paths[1] = [3,5,7,10]
paths[2] = [i for i in range(2, 11)]
paths[3] = [2,5,7,10]
paths[4] = [i for i in range(1, 11)]
paths[5] = [5]
paths[6] = [i for i in range(1, 11)]
paths[7] = [0,1,5,10]
paths[8] = [0,5,10]
paths[9] = [0,1,5,7,10]
paths[10] = [0,5,6,7,10]

for row_index in range(0, 11):
  for column_index in paths[row_index]:
    rewards[row_index, column_index] = -1.
    
#objective
rewards[0, 5] = 100

for row in rewards:
  print(row)

[-100 -100 -100   -1   -1  100   -1   -1 -100 -100   -1]
[-100 -100 -100   -1 -100   -1 -100   -1 -100 -100   -1]
[-100 -100   -1   -1   -1   -1   -1   -1   -1   -1   -1]
[-100 -100   -1 -100 -100   -1 -100   -1 -100 -100   -1]
[-100   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1]
[-100 -100 -100 -100 -100   -1 -100 -100 -100 -100 -100]
[-100   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1]
[  -1   -1 -100 -100 -100   -1 -100 -100 -100 -100   -1]
[  -1 -100 -100 -100 -100   -1 -100 -100 -100 -100   -1]
[  -1   -1 -100 -100 -100   -1 -100   -1 -100 -100   -1]
[  -1 -100 -100 -100 -100   -1   -1   -1 -100 -100   -1]


In [7]:
def is_terminal_state(current_row, current_column):
  if rewards[current_row, current_column] == -1.:
    return False
  else:
    return True

In [8]:
def get_start():
  current_row = np.random.randint(area_rows)
  current_column = np.random.randint(area_columns)
 
  while is_terminal_state(current_row, current_column):
    current_row = np.random.randint(area_rows)
    current_column = np.random.randint(area_columns)
  return current_row, current_column

In [9]:
def get_next_action(current_row, current_column, epsilon):
  if np.random.random() < epsilon:
    return np.argmax(q_table[current_row, current_column])
  else:
    return np.random.randint(4)

In [10]:
def get_next_location(current_row, current_column, movement_action):
  new_row_index = current_row
  new_column_index = current_column
  if actions[movement_action] == 'up' and current_row > 0:
    new_row_index -= 1
  elif actions[movement_action] == 'down' and current_row < area_rows - 1:
    new_row_index += 1
  elif actions[movement_action] == 'left' and current_column > 0:
    new_column_index -= 1  
  elif actions[movement_action] == 'right' and current_column < area_columns - 1:
    new_column_index += 1
  return new_row_index, new_column_index

In [11]:
def get_shortest_path(start_row_index, start_column_index):
  if is_terminal_state(start_row_index, start_column_index):
    return []
  else:
    current_row, current_column = start_row_index, start_column_index
    shortest_path = []
    shortest_path.append([current_row, current_column])
   
    while not is_terminal_state(current_row, current_column):
      movement_action = get_next_action(current_row, current_column, 1.)
      current_row, current_column = get_next_location(current_row, current_column, movement_action)
      shortest_path.append([current_row, current_column])
    return shortest_path

## Training

In [12]:
epsilon = 0.9 #Chance to choose the best action and not ramdom
gamma = 0.9 #discount factor
learning_rate = 0.55 


for iteration in range(1000):
  row_index, column_index = get_start()

  while not is_terminal_state(row_index, column_index):
    #Choose next movement and store old position
    movement_action = get_next_action(row_index, column_index, epsilon)
    old_row_index, old_column_index = row_index, column_index
    row_index, column_index = get_next_location(row_index, column_index, movement_action)
    
    #recieve reward 
    reward = rewards[row_index, column_index]
    old_q_table = q_table[old_row_index, old_column_index, movement_action]
    temporal_difference = reward + (gamma * np.max(q_table[row_index, column_index])) - old_q_table

    #update the Q-value and table
    new_q_value = old_q_table + (learning_rate * temporal_difference)
    q_table[old_row_index, old_column_index, movement_action] = new_q_value

print('Finished!')

Finished!


In [13]:
for row in rewards:
  print(row)

[-100 -100 -100   -1   -1  100   -1   -1 -100 -100   -1]
[-100 -100 -100   -1 -100   -1 -100   -1 -100 -100   -1]
[-100 -100   -1   -1   -1   -1   -1   -1   -1   -1   -1]
[-100 -100   -1 -100 -100   -1 -100   -1 -100 -100   -1]
[-100   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1]
[-100 -100 -100 -100 -100   -1 -100 -100 -100 -100 -100]
[-100   -1   -1   -1   -1   -1   -1   -1   -1   -1   -1]
[  -1   -1 -100 -100 -100   -1 -100 -100 -100 -100   -1]
[  -1 -100 -100 -100 -100   -1 -100 -100 -100 -100   -1]
[  -1   -1 -100 -100 -100   -1 -100   -1 -100 -100   -1]
[  -1 -100 -100 -100 -100   -1   -1   -1 -100 -100   -1]


In [19]:
print(get_shortest_path(4, 2)) 

[[4, 2], [4, 3], [4, 4], [4, 5], [3, 5], [2, 5], [1, 5], [0, 5]]
